In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import tokenize
import matplotlib.pyplot as plt
import html
import re
import random
import nltk
nltk.download('punkt')
#if there is a problem in importing transformer
# please install using pip from the code below
import requests
from bs4 import BeautifulSoup
import os.path
from dateutil import parser
import pandas as pd
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [32]:
#!pip install trafilatura

In [2]:
import trafilatura
downloaded = trafilatura.fetch_url('https://www.nginx.com/blog/now-worlds-1-web-server-nginx-looks-forward-to-even-brighter-future/?utm_medium=email&utm_source=nginxdb&utm_campaign=ww-nx_mad&utm_content=bg&mkt_tok=NjUzLVNNQy03ODMAAAF-ItKyIbnEbPZ46g8i-GcuhvIcWazwBd-R9JPPX4bEN86nCcd-MGGs_n6sdIKJxFPlCMDPfZqKqlwSe6e_543i9JJJO6t1VITuU24-JIsNBDIbqgm7')
trafilatura.extract(downloaded)
# outputs main content and comments as plain text ...
from trafilatura import bare_extraction
#bare_extraction(downloaded)

{'author': 'Tony Mauro',
 'categories': ['Blog,News'],
 'comments': '',
 'date': '2021-06-10',
 'description': "NGINX recently became the most popular web server in the world, according to W3Techs. We're profoundly grateful to the NGINX community, who've brought us to this milestone, and look forward to providing even more tools to help you optimize delivery of your modern and cloud-native apps.",
 'fingerprint': None,
 'hostname': 'nginx.com',
 'id': None,
 'license': None,
 'sitename': 'NGINX',
 'tags': [],
 'text': 'Last month, NGINX passed a major milestone. W3Techs announced that after many years of steady growth in market share, NGINX is now the most popular web server in the world, edging out Apache HTTP Server. Of course we’re thrilled to be recognized for our years of innovation and hard work, but even more than that we’re profoundly grateful to you, the members of the NGINX community, who’ve brought us to this milestone by trusting us to secure and deliver your websites and a

In [3]:
#Calculating the number of questions as needed depending upon the length of text
import math
mohack = []
text = bare_extraction(downloaded)['text']
mohack=text.splitlines()
number_of_threads = math.trunc(len(text)/3200)
number_of_threads

2

In [4]:
#Installing a libraty to generate question using transformer n-to-n model
!git clone https://github.com/amontgomerie/question_generator

fatal: destination path 'question_generator' already exists and is not an empty directory.


In [5]:
%cd question_generator/

/content/question_generator


In [6]:
%ls

articles/  LICENSE       questiongenerator.py  requirements.txt  setup.py
examples/  __pycache__/  README.md             run_qg.py         training/


In [31]:
# !pip install transformers
# !pip install sentencepiece

In [7]:
#importing the library for question generation
from questiongenerator import QuestionGenerator
qg = QuestionGenerator()

In [8]:

import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
assert device == torch.device('cuda')
device

device(type='cuda')

In [9]:
#Generating question and answer with respect to the length of the text
#number of question would result number of threads
lp = qg.generate(text, num_questions=number_of_threads,answer_style='all')

Generating questions...



/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:191: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


Evaluating QA pairs...



In [10]:
#Now appending the questions in the text and then calculating cosine similarity with all 
#the paragraphs and then extracting those paragraphs to build summary
for i in range(number_of_threads):
  mohack.append(lp[i]['question'])

import gensim
# upgrade gensim if you can't import softcossim
from gensim.matutils import softcossim 
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess
print(gensim.__version__)
#> '3.6.0'

# Download the FastText model#
#


3.6.0


In [12]:
#using fasttext model to calculate similarity
fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')

In [13]:
# Prepare a dictionary and a corpus.
dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in mohack])

# Prepare the similarity matrix
similarity_matrix = fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)

In [14]:
#create bag of words model for all the paragraphs 
sentences = []
for i in range(len(mohack)):
  sentences.append(dictionary.doc2bow(simple_preprocess(mohack[i])))
print(softcossim(sentences[0], sentences[1], similarity_matrix))

0.573541026789918


In [16]:
#we compute the cosim=ne similarity matrix between all the paragraphs and qiestions
import numpy as np
import pandas as pd

def create_soft_cossim_matrix(sentences):
    len_array = np.arange(len(sentences))
    xx, yy = np.meshgrid(len_array, len_array)
    cossim_mat = pd.DataFrame([[round(softcossim(sentences[i],sentences[j], similarity_matrix) ,2) for i, j in zip(x,y)] for y, x in zip(xx, yy)])
    return cossim_mat

matrix =create_soft_cossim_matrix(sentences)

In [26]:
#now we will extract the paragraphs which are related to question and then 
#generate thier summary 
context = {}
for i in range(1,number_of_threads + 1):
  lm = []
  for j in range(0,len(matrix.iloc[-i])-number_of_threads):
    #only strong similarity connection paragraphs would be appended
    if matrix.iloc[-i][j] >= 0.65 and matrix.iloc[-i][j] < 1.00 :
      lm.append(j) 
    context[lp[-i]['question']] = lm 
context

{'What is the main reason why NGINX is the #1 server?': [],
 'What was the motivation for NGINX to become the most popular web server in the world?': [0,
  4,
  5,
  14,
  18]}

In [33]:
from transformers import BartTokenizer, BartModel
import torch
#for the process of abstractive summmarization I have used Bart-cnn from hugging face as pretrained model
from typing import List
#bart-cnn uses encoder and decoder for summarization
#but it's only effective for tokens less than 1300 around
from transformers import BartForConditionalGeneration
def old_summarization_pipeline(text: List[str]) -> List[str]:
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
    input_ids = tokenizer.batch_encode_plus(text, truncation=True, padding=True, return_tensors='pt', max_length=1024)['input_ids']
    summary_ids = model.generate(input_ids)
    summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=False) for s in summary_ids]
    return summaries

In [27]:
#Now cleaning stuff annd then the last part i.e. summarization
for i,j in context.items():
  lm = [] 
  for k in range(len(j)):
    lm.append(old_summarization_pipeline([mohack[j[k]]]))
    context[i] = lm


In [29]:
twitterthread = {}
finalsummary = []
for i,j in context.items():
 lmo = []
 for k in range(len(j)):
    lmo = j[k][0].split(".")
    new_list = [elem for elem in lmo if elem.strip()]
    finalsummary.append(new_list)
    dict1= {}
    d = 0
    for l in range(len(finalsummary)):
      for m in range(len(finalsummary[l])):
        d = d+1
        dict1[d] = finalsummary[l][m]
    twitterthread[i] = dict1 
  

In [30]:
twitterthread

{'What was the motivation for NGINX to become the most popular web server in the world?': {1: 'NGINX is now the most popular web server in the world, edging out Apache HTTP Server',
  2: ' W3Techs announced that after many years of steady growth in market share, NGINX has overtaken Apache',
  3: " We're thrilled to be recognized for our years of innovation and hard work, but even more than that we’re profoundly grateful to you",
  4: "The original motivation for creating NGINX wasn't nearly so grand",
  5: ' Igor Sysoev was trying to solve a problem at work – his web servers were having trouble keeping up with ever‑increasing numbers of requests',
  6: ' The challenge was referred to at the time as the C10K problem – handling 10,000 simultaneous client connections',
  7: ' NGINX is a lightweight, scalable, and powerful distributed system',
  8: ' It was created by Igor Kuznetsov in 2004 and is still at the heart of many websites today',
  9: ' Kuznetskov: "Igor developed an event‑drive